In [1]:
import pymssql
import requests
from bs4 import BeautifulSoup


db_settings = {
    "host": "127.0.0.1",
    "user": "sa",
    "password": "",
    "database": "ncu_database",
    "charset": "utf8"
}

In [ ]:
def find_stock(url, start, end):
    try:
        conn = pymssql.connect(**db_settings)
        with conn.cursor() as cursor:
            command = "INSERT INTO [dbo].[stock_list] (id, name, type, catogory) VALUES (%s, %s, %s, %s)"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")

            result = soup.select("table td b")
            for tr in result: # 在所有的粗體標籤中找到我們需要的
                if(tr.text.strip() == start):
                    start_td = tr.find_parent("tr")
                elif(tr.text.strip() == end):
                    end_td = tr.find_parent("tr")

            list_td = start_td.find_next("tr") # 找出下一個tr標籤
            while(list_td != end_td):
                child_list = list_td.find_all('td') # 列出tr標籤中的所有td標籤
                stock_id, stock_name = child_list[0].text.split("　")
                cursor.execute(command, (stock_id, stock_name, child_list[3].text, child_list[4].text))
                list_td = list_td.find_next("tr")
            conn.commit()
    except Exception as e:
    #    print(e)
        error_class = e.__class__.__name__ #取得錯誤類型
        detail = e.args[0] #取得詳細內容
        cl, exc, tb = sys.exc_info() #取得Call Stack
        lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
        fileName = lastCallStack[0] #取得發生的檔案名稱
        lineNum = lastCallStack[1] #取得發生的行號
        funcName = lastCallStack[2] #取得發生的函數名稱
        errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
        print(errMsg)

In [ ]:
find_stock("https://isin.twse.com.tw/isin/C_public.jsp?strMode=4", "股票", "特別股")
find_stock("https://isin.twse.com.tw/isin/C_public.jsp?strMode=2", "股票", "上市認購(售)權證")